In [61]:
from nltk.corpus import gutenberg, brown, reuters
import nltk

from collections import defaultdict
from pprint import pprint

import random
%pprint

Pretty printing has been turned ON


In [2]:
for fileid in gutenberg.fileids():
    num_chars = len(gutenberg.raw(fileid))
    num_words = len(gutenberg.words(fileid))
    num_sents = len(gutenberg.sents(fileid))
    num_vocab = len(set(w.lower() for w in gutenberg.words(fileid)))
    print(round(num_chars/num_words), round(num_words/num_sents), round(num_words/num_vocab), fileid)

5 25 26 austen-emma.txt
5 26 17 austen-persuasion.txt
5 28 22 austen-sense.txt
4 34 79 bible-kjv.txt
5 19 5 blake-poems.txt
4 19 14 bryant-stories.txt
4 18 12 burgess-busterbrown.txt
4 20 13 carroll-alice.txt
5 20 12 chesterton-ball.txt
5 23 11 chesterton-brown.txt
5 18 11 chesterton-thursday.txt
4 21 25 edgeworth-parents.txt
5 26 15 melville-moby_dick.txt
5 52 11 milton-paradise.txt
4 12 9 shakespeare-caesar.txt
4 12 8 shakespeare-hamlet.txt
4 12 7 shakespeare-macbeth.txt
5 36 12 whitman-leaves.txt


In [77]:
fileids = None
# fileids = brown.fileids()[:250]
sents_brown = list(brown.sents(fileids=fileids))

fileids = None
# fileids = ['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', '']
sents_gutenberg = list(gutenberg.sents(fileids=fileids))

sents = sents_brown + sents_gutenberg

print('Sentences:', len(sents))

Sentences: 155892


In [80]:
for i in range(len(sents)):
    sents[i] = tuple(['<START>'] + sents[i] + ['<END>'])
    
sents_indx = {i: s for i, s in enumerate(sents)}
indx_sents = {s: i for i, s in enumerate(sents)}

TypeError: can only concatenate list (not "tuple") to list

In [90]:
words_sents = defaultdict(set)

for i, s in sents_indx.items():
    for w in s:
        words_sents[w].add(i)

print('Unique words:', len(words_sents))

Unique words: 84544


In [81]:
def word_sents(word):
    return [sents_indx[i] for i in words_sents.get(word, set())]


In [91]:
def generate_text_sents(words):
    if isinstance(words, str):
        words = [w.strip() for w in words.split(',')]
    
    text = []

    for w in words:
        w_sents = word_sents(w)

        if w_sents:
            s = random.choice(w_sents)
        else:
            s = tuple()

        text.append(' '.join(s))
    
    return text


In [92]:
pprint(generate_text_sents('try, love, hate'))

['<START> When driving rain or mist socked in one valley , Fogg would '
 'chandelle up and over to reverse course and try another one , ranging from '
 'the Ottauquechee up to Danville in search of safe passage through the '
 'mountain passes . <END>',
 '<START> 19 : 34 But the stranger that dwelleth with you shall be unto you as '
 'one born among you , and thou shalt love him as thyself ; for ye were '
 'strangers in the land of Egypt : I am the LORD your God . <END>',
 '<START> The swords caught on each other with a dreadful clang and jar , full '
 'of the old energy and hate ; and at once plunged and replunged . <END>']


In [122]:
import random

class Dictogram(dict):
    def __init__(self, iterable=None):
        # Инициализируем наше распределение как новый объект класса, 
        # добавляем имеющиеся элементы
        super(Dictogram, self).__init__()
        self.types = 0  # число уникальных ключей в распределении
        self.tokens = 0  # общее количество всех слов в распределении
        if iterable:
            self.update(iterable)

    def update(self, iterable):
        # Обновляем распределение элементами из имеющегося 
        # итерируемого набора данных
        for item in iterable:
            if item in self:
                self[item] += 1
                self.tokens += 1
            else:
                self[item] = 1
                self.types += 1
                self.tokens += 1

    def count(self, item):
        # Возвращаем значение счетчика элемента, или 0
        if item in self:
            return self[item]
        return 0

    def return_random_word(self):
        random_key = random.sample(self, 1)
        # Другой способ:
        # random.choice(histogram.keys())
        return random_key[0]

    def return_weighted_random_word(self):
        # Сгенерировать псевдослучайное число между 0 и (n-1),
        # где n - общее число слов
        random_int = random.randint(0, self.tokens-1)
        index = 0
        list_of_keys = list(self.keys())
        # вывести 'случайный индекс:', random_int
        for i in range(0, self.types):
            index += self[list_of_keys[i]]
            # вывести индекс
            if(index > random_int):
                # вывести list_of_keys[i]
                return list_of_keys[i]
            

def make_markov_model(data):
    markov_model = dict()

    for i in range(0, len(data)-1):
        if data[i] in markov_model:
            # Просто присоединяем к уже существующему распределению
            markov_model[data[i]].update([data[i+1]])
        else:
            markov_model[data[i]] = Dictogram([data[i+1]])
            
    return markov_model

def make_higher_order_markov_model(order, data):
    markov_model = dict()

    for i in range(0, len(data)-order):
        # Создаем окно
        window = tuple(data[i: i+order])
        # Добавляем в словарь
        if window in markov_model:
            # Присоединяем к уже существующему распределению
            markov_model[window].update([data[i+order]])
        else:
            markov_model[window] = Dictogram([data[i+order]])
    return markov_model

In [166]:
import random
from collections import deque
import re

def generate_random_start(model):
    # Чтобы сгенерировать любое начальное слово, раскомментируйте строку:
    # return random.choice(model.keys())

    # Чтобы сгенерировать "правильное" начальное слово, используйте код ниже:
    # Правильные начальные слова - это те, что являлись началом предложений в корпусе
    if '<END>' in model:
        seed_word = '<END>'
        while seed_word == '<END>':
            seed_word = model['<END>'].return_weighted_random_word()
        return seed_word
    return random.choice(list(model.keys()))


def generate_random_sentence(length, markov_model):
    current_word = generate_random_start(markov_model)
    sentence = [current_word]
    for i in range(0, length):
        current_dictogram = markov_model[current_word]
        random_weighted_word = current_dictogram.return_weighted_random_word()
        current_word = random_weighted_word
        sentence.append(current_word)
#     sentence[0] = sentence[0].capitalize()
#     return ' '.join(sentence) + '.'
    return sentence

In [167]:
n = 1

sents_ngrams = []

for s in sents:
    sents_ngrams.append(
        nltk.ngrams(s, n=n, pad_left=False, pad_right=False, left_pad_symbol='START', right_pad_symbol='END')
    )
    
text_ngrams = [ngram for sent_ngrams in sents_ngrams for ngram in sent_ngrams]

In [169]:
m_model = make_markov_model([str(ngr[0]) for ngr in text_ngrams[:1000000]])

'f'

In [184]:
s = generate_random_sentence(20, m_model)
print(' '.join(s))
sent = []

for ngr in s:
    for w in ngr:
        if w not in sent:
            sent.append(w)


<START> Does a people into visiting `` places cooperative must have restricted to put . <END> <START> Other scientific explanations are


In [113]:
print('N-grams:', len(text_ngrams))
text_ngrams[:100]

N-grams: 4406545


[(None, None, '<START>'),
 (None, '<START>', 'The'),
 ('<START>', 'The', 'Fulton'),
 ('The', 'Fulton', 'County'),
 ('Fulton', 'County', 'Grand'),
 ('County', 'Grand', 'Jury'),
 ('Grand', 'Jury', 'said'),
 ('Jury', 'said', 'Friday'),
 ('said', 'Friday', 'an'),
 ('Friday', 'an', 'investigation'),
 ('an', 'investigation', 'of'),
 ('investigation', 'of', "Atlanta's"),
 ('of', "Atlanta's", 'recent'),
 ("Atlanta's", 'recent', 'primary'),
 ('recent', 'primary', 'election'),
 ('primary', 'election', 'produced'),
 ('election', 'produced', '``'),
 ('produced', '``', 'no'),
 ('``', 'no', 'evidence'),
 ('no', 'evidence', "''"),
 ('evidence', "''", 'that'),
 ("''", 'that', 'any'),
 ('that', 'any', 'irregularities'),
 ('any', 'irregularities', 'took'),
 ('irregularities', 'took', 'place'),
 ('took', 'place', '.'),
 ('place', '.', '<END>'),
 ('.', '<END>', None),
 ('<END>', None, None),
 (None, None, '<START>'),
 (None, '<START>', 'The'),
 ('<START>', 'The', 'jury'),
 ('The', 'jury', 'further'),
 ('j